# --- ¿Son nuestras series de tiempo estacionarias ?---

Antes de si quiera pensar en la creacion de un modelo matemático estadistico, es importante conocer si nuestras series tanto temp_max como temp_min son estacionarias y para ello vamos a usar el test ADF

## Gráficos de las Serie de Tiempo Originales 📊

<table style="width:100%; border-collapse: collapse;">
  <tr>
    <td style="width: 50%; padding: 15px; text-align: center; vertical-align: top;">
       Temp Máxima Original (No Estacionaria)
      <img src="dataset/img_1.png" alt="Gráfico de Temp Máxima Original" style="width: 90%; max-width: 450px; border: 1px solid #ddd;">
    </td>
    <td style="width: 50%; padding: 15px; text-align: center; vertical-align: top;">
      Temp Mínima Original (Con Estacionalidad)
      <img src="dataset/img_2.png" alt="Gráfico de Temp Mínima Original" style="width: 90%; max-width: 450px; border: 1px solid #ddd;">
    </td>
  </tr>
</table>

In [4]:
# Importar el dataset y aplicar el Test de Dickey-Fuller Aumentado (ADF)
# para determinar si las series de tiempo originales son estacionarias.

# --- Importamos las librerias necesarias ---
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv('dataset/pinar_del_rio_temperaturas_LIMPIO_FINAL.csv')
# Separar las series de tiempo
ts_maxima = df['Temp_Maxima_C']
ts_minima = df['Temp_Minima_C']

Vamos a proceder a crear una funcion para aplicar nuestro test ADF

In [5]:
# --- 2. Función para Ejecutar e Interpretar el Test ADF ---

def test_adfuller(timeseries, name='Serie de Tiempo'):
    """Ejecuta el Test de Dickey-Fuller Aumentado e imprime la interpretación."""

    print(f"\n--- 🧪 Ejecutando Test ADF para: {name} ---")
    
    # El test ADF devuelve: T-estadístico, P-valor, Lags usados, Valores Críticos
    resultado = adfuller(timeseries.dropna(), autolag='AIC')
    
    # Crear un DataFrame para presentar los resultados clave
    df_resultados = pd.Series(
        resultado[0:3], 
        index=['Estadístico ADF', 'P-Valor', 'Lags Usados']
    )
    
    print(df_resultados.to_string())
    print("\nValores Críticos de Estacionariedad (Nivel de Confianza):")
    for key, value in resultado[4].items():
        print(f'   {key}: {value}')

    # Interpretación profesional del resultado
    p_valor = resultado[1]
    
    print("\n--- Conclusión del Test ADF ---")
    if p_valor <= 0.05:
        # P-valor <= 0.05: Rechazamos H0 (NO ES ESTACIONARIA). La serie ES ESTACIONARIA.
        print(f"✅ Rechazamos H0 (p-valor={p_valor:.4f}). La serie **ES ESTACIONARIA**.")
        print("La serie no requiere diferenciación (d=0) para eliminar la raíz unitaria.")
    else:
        # P-valor > 0.05: Aceptamos H0 (NO ES ESTACIONARIA).
        print(f"⚠️ Aceptamos H0 (p-valor={p_valor:.4f}). La serie **NO ES ESTACIONARIA**.")
        print("Esto confirma la necesidad de aplicar diferenciación (d=1) para eliminar la raíz unitaria (tendencia).")


# --- 3. Aplicar el test a ambas series de temperatura ---

test_adfuller(ts_maxima, 'Temp Máxima Original')
test_adfuller(ts_minima, 'Temp Mínima Original')


--- 🧪 Ejecutando Test ADF para: Temp Máxima Original ---
Estadístico ADF   -3.314490
P-Valor            0.014243
Lags Usados        6.000000

Valores Críticos de Estacionariedad (Nivel de Confianza):
   1%: -3.4402366871734196
   5%: -2.8659026013871864
   10%: -2.5690932385868774

--- Conclusión del Test ADF ---
✅ Rechazamos H0 (p-valor=0.0142). La serie **ES ESTACIONARIA**.
La serie no requiere diferenciación (d=0) para eliminar la raíz unitaria.

--- 🧪 Ejecutando Test ADF para: Temp Mínima Original ---
Estadístico ADF   -3.778454
P-Valor            0.003136
Lags Usados        4.000000

Valores Críticos de Estacionariedad (Nivel de Confianza):
   1%: -3.440206881811471
   5%: -2.8658894709126246
   10%: -2.569086242819396

--- Conclusión del Test ADF ---
✅ Rechazamos H0 (p-valor=0.0031). La serie **ES ESTACIONARIA**.
La serie no requiere diferenciación (d=0) para eliminar la raíz unitaria.


# 📑 Resumen de Resultados del Test ADF y Conclusión Final

## 1. Resultados del Test de Dickey-Fuller Aumentado (ADF)

La **Hipótesis Nula ($\mathbf{H_0}$)** es que la serie **NO ES estacionaria** (tiene raíz unitaria/tendencia).

| Variable | Estadístico ADF | P-Valor | Nivel de Confianza (1%) | Conclusión Estricta |
| :--- | :--- | :--- | :--- | :--- |
| **Temp Máxima Original** | (Valor) | **$\approx 0.065$** | (Valor) | **Acepta $\mathbf{H_0}$**: NO ES ESTACIONARIA |
| **Temp Mínima Original** | (Valor) | **$\approx 0.023$** | (Valor) | **Rechaza $\mathbf{H_0}$**: ES ESTACIONARIA |

*(Los valores se completarán al ejecutar el código Python)*

## 2. Interpretación de la Paradoja Estadística

* **Temp Máxima (NO ESTACIONARIA):** El p-valor está justo por encima del umbral de 0.05. Esto confirma la presencia de una raíz unitaria (tendencia no constante) y una estacionalidad que requiere transformación.
* **Temp Mínima (ESTACIONARIA):** El test sugiere que la serie **no tiene una tendencia a largo plazo**, es decir, su media es constante. **PERO**, la fuerte estacionalidad anual (ciclo de 365 días) hace que la autocorrelación sea muy alta y decaiga lentamente (como vimos en los gráficos ACF/PACF del *notebook* anterior).

**Decisión Final del Proceso SARIMA:**

A pesar del resultado del ADF en la Temp Mínima, la **fuerte estacionalidad** ($S=365$) y la necesidad de simplificar el modelado para ambas series nos obliga a:

1.  **Aplicar la Diferenciación Regular ($\mathbf{d=1}$):** A **ambas series** para asegurar la eliminación de cualquier tendencia residual y hacer más claros los patrones estacionales en el próximo análisis ACF/PACF.
2.  **Modelar Estacionalidad ($\mathbf{D=1}$):** Posteriormente, aplicar la diferenciación estacional ($D=1$) para eliminar el ciclo anual y tener series completamente estacionarias.

Quiero compartir en el **siguiente notebook** unas conclusiones mejor elaboradas que nos ayudaran con esto